https://www.kaggle.com/c/jane-street-market-prediction/data?select=train.csv

https://www.kaggle.com/carlmcbrideellis/jane-street-eda-of-day-0-and-feature-importance

https://www.analyticsvidhya.com/blog/2016/03/complete-guide-parameter-tuning-xgboost-with-codes-python/

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
import plotly.express as px
import plotly.graph_objects as go
colorMap = sns.light_palette("blue", as_cmap=True)
import warnings
warnings.filterwarnings('ignore')
colorMap = sns.light_palette("blue", as_cmap=True)
import missingno as msno

internet = True
debug = False

In [ ]:
if internet:
    #install datatable
    !pip install datatable > /dev/null
    import datatable as dt

In [ ]:
%%time

if internet:
    train_data_datatable = dt.fread('../input/jane-street-market-prediction/train.csv')
    train_data = train_data_datatable.to_pandas()
else:
    train_data = pd.read_csv('../input/jane-street-market-prediction/train.csv')

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, LSTM, Bidirectional
from keras.regularizers import l2
from keras.initializers import GlorotNormal
from keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler

preprocessing

In [ ]:
features_csv = pd.read_csv('../input/jane-street-market-prediction/features.csv')

In [ ]:
x_data = np.array([])
number_of_tags = 28
for i in range(number_of_tags+1):
    tag_col = features_csv[f'tag_{i}']
    true_index = tag_col.to_numpy().nonzero()
    arr = pd.Series(train_data[f'feature_{true_index[0][0]}']).to_numpy().reshape(-1, 1)
    for j in range(1, len(true_index[0])):
        tmp = train_data[f'feature_{true_index[0][j]}'].to_numpy()
        arr = np.insert(arr, 0, tmp, axis=1)
    
    val = np.nanmean(arr, axis=1)
    
    if i == 0:
        x_data = val.reshape(-1, 1)
    else:
        x_data = np.insert(x_data, i, val, axis=1)
    
    print(i)

In [ ]:
x_data = np.insert(x_data, 0, train_data['feature_0'].to_numpy(), axis=1)

In [ ]:
weights = pd.Series(train_data['weight']).to_numpy()
resp = pd.Series(train_data['resp'])
resp_np = resp.to_numpy()
resp_np = resp_np.reshape(-1, 1)

for i in range(1, 5):
    tmp_resp = pd.Series(train_data[f'resp_{i}'])
    resp += tmp_resp
    resp_np = np.insert(resp_np, 0, tmp_resp.to_numpy(), axis=1)

In [ ]:
#np_resp = resp.to_numpy().reshape(-1, 1)

resp_scaler = MinMaxScaler(feature_range=(-1,1))
feature_scaler = MinMaxScaler(feature_range=(-1,1))

xt_data = np.nan_to_num(x_data)
x_train = xt_data.copy()
#x_train = feature_scaler.fit_transform(xt_data)
#y_train = np.count_nonzero(resp_np > 0, axis=1)/5

y_train = (np.count_nonzero(resp_np > 0, axis = 1) >= 3)
y_train = y_train.reshape(-1, 1)

In [ ]:
val_x_train = x_train[2300000:len(x_train)]
val_y_train = y_train[2300000:len(y_train)]
x_train = x_train[:2300000]
y_train = y_train[:2300000]

In [ ]:
import xgboost as xgb
from sklearn.model_selection import GridSearchCV
"""
clf = xgb.XGBClassifier(
    n_estimators=4000,
    max_depth=11,
    learning_rate=0.005,
    subsample=0.9,
    colsample_bytree=0.7,
    missing=-999,
    random_state=2020,
    tree_method='gpu_hist'
)
"""

clf = xgb.XGBClassifier(
    n_estimators=50000,
    max_depth=6,
    learning_rate=0.0005,
    tree_method='gpu_hist',
    subsample=0.7,
    colsample_bytree=0.7,
    objective='binary:logistic',
    seed=42
    )

%time clf.fit(x_train, y_train, early_stopping_rounds=50, eval_set=[(val_x_train, val_y_train)], verbose = 0)

In [ ]:
np.count_nonzero(clf.predict(val_x_train) == val_y_train.reshape(-1))/len(val_x_train)

In [ ]:
"""
if not debug:
    model = Sequential()
    model.add(Dense(units=30, kernel_regularizer=l2(1e-4), kernel_initializer=GlorotNormal(),
                   activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=64, kernel_regularizer=l2(1e-4), kernel_initializer=GlorotNormal(),
                   activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(units=64, kernel_regularizer=l2(1e-4), kernel_initializer=GlorotNormal(),
                   activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer=Adam(learning_rate=0.00001))
    model.fit(x_train, y_train, epochs=2, batch_size=32, 
              validation_data=(x_train[:1000], y_train[:1000]))
else:
    model = Sequential()
    model.add(Dense(units=16, kernel_regularizer=l2(1e-4), kernel_initializer=GlorotNormal()))
    model.add(Dense(1))
    model.compile(loss='mse', optimizer=Adam(learning_rate=0.0001))
    model.fit(x_train, y_train, epochs=1, batch_size=128, validation_data=(x_train[:1000], y_train[:1000]))
"""

In [ ]:
f = clf.predict(x_train[0].reshape(1, -1))
f[0]

In [ ]:
import janestreet
env = janestreet.make_env() # initialize the environment
iter_test = env.iter_test() # an iterator which loops over the test set

In [ ]:
import time

start = time.perf_counter()

if not debug:
    index_list = []
    for i in range(number_of_tags+1):
        index_list.append(features_csv[f'tag_{i}'].to_numpy().nonzero())

    for (test_df, sample_prediction_df) in iter_test:
        if test_df.iloc[0]['weight'] <= 0:
            sample_prediction_df.action = 0
            env.predict(sample_prediction_df)
            continue

        val_list = np.array(test_df.iloc[0])[3:]
        pred = np.zeros([2+number_of_tags])
        pred[0] = test_df['feature_0'].to_numpy()[0]
        for i in range(number_of_tags+1):
            val = 0
            cnt = 0
            for j in index_list[i][0]:
                feature_num = val_list[j-1]
                if np.isnan(feature_num):
                    continue
                val += feature_num
                cnt += 1

            if cnt != 0:
                val /= cnt

            pred[i+1] = val

        pred = pred.reshape(1, -1)
        action = clf.predict(pred)
        sample_prediction_df.action = 1 if action[0] == True else 0 #make your 0/1 prediction here
        env.predict(sample_prediction_df)

    print(time.perf_counter()-start)